## First Aproach

- Simular $n$ termoeléctricas ($t_i$ es el valor de generación máxima de la termoeléctrica $i$)
- El día 0 funcionan todas
- Si todas funcionan Oferta $\geq$ Demanda
- Simular tiempo de rotura (Weibull)
- Todas se pueden reparar a la vez
- Tiempo de reparación Lognormal
- Existen $m$ circuitos ($c_j$ es el valor de demanda del circuito $j$)
- $\delta_{ik}$ es un parámetro binario que expresa si la termoeléctrica $i$ está activa el día $k$
- $\alpha_{jk}$ es un parámetro binario que expresa si el circuito $j$ está activo el día $k$
- Existe capacidad de almacenamiento ($a_k$ es la capacidad de almacenamiento del sistema en el día $k$)
- $a_0 = 0$
- $a_k = \sum_{i=1}^{n} t_i\delta_{ik} + a_{k-1} - \sum_{j=1}^{m} c_j\alpha_{jk}, a_k \geq 0$
- No simularemos mantenimientos planificados, solo reparación por rotura
- No tendremos en cuenta fallas internas de los circuitos
- No habrán circuitos que dependan de termoeléctricas específicas
- La afectación a los circuitos se hará de manera proporcional (con una proporción que definiremos) y rotativa, cuando sea necesario
- Al final de la simulación calcularemos una afectación (fórmula por determinar)

In [1]:
from abc import ABC, abstractmethod
import math

class Handler(ABC):

    @abstractmethod
    def decision(self, offer, t, circuits, circuits_state):
        pass

class RRHandler(Handler):

    current_index = 0

    def __init__(self) -> None:
        super().__init__()

    def decision(self, offer, t, circuits, circuits_state):
        
        demand = sum(circuits)

        while(demand > offer):
            circuits_state[t][self.current_index] -= math.floor(0.25 * circuits[self.current_index])
            demand -= math.floor(0.25 * circuits[self.current_index])

            self.current_index += 1
            self.current_index %= len(circuits)

            if circuits_state[t][self.current_index] < 0:
                raise Exception("Circuit fall below 0")

        return demand
        
        

In [2]:
import probabilistic_models as pm
import numpy as np
import random

def simulate(N, M, K, handler:Handler ,seed=0):

    if seed != 0:
        random.seed(seed)

    thermoelectrics = [random.randint(500, 1000) for _ in range(N)]
    thermoelectrics_state = [[1 for _ in range(N)]]
    circuits = [random.randint(200, 600) for _ in range(M)]
    circuits_state = [[x for x in circuits]]
    leave_service = [[] for _ in range(K)]
    re_enter_service = [[] for _ in range(K)]
    balance = 0
    demand = sum(x for x in circuits)

    for i in range(N):
        pm.break_thermoelectric(leave_service, i, 0, N)

    for t in range(1, K):
        circuits_state.append([x for x in circuits_state[t - 1]])
        thermoelectrics_state.append([x for x in thermoelectrics_state[t - 1]])
        
        for thermoelectric in leave_service[t]:
            thermoelectrics_state[t][thermoelectric] = 0
            pm.repair_thermoelectric(re_enter_service, thermoelectric, t, N)

        for thermoelectric in re_enter_service[t]:
            thermoelectrics_state[t][thermoelectric] = 1
            pm.break_thermoelectric(leave_service, thermoelectric, t, N)
            
        offer = sum(x * y for x, y in zip(thermoelectrics, thermoelectrics_state[t])) + balance
        demand = handler.decision(offer, t, circuits, circuits_state)

        balance = offer - demand
    
    return [thermoelectrics_state, circuits_state]

In [3]:
thermoelectrics, circuits = simulate(20, 15, 60, RRHandler())

for i in range(365):
    print(thermoelectrics[i])

Thermoelectric 0 out of service on day 1
Thermoelectric 1 out of service on day 1
Thermoelectric 2 out of service on day 1
Thermoelectric 3 out of service on day 3
Thermoelectric 4 out of service on day 1
Thermoelectric 5 out of service on day 3
Thermoelectric 6 out of service on day 1
Thermoelectric 7 out of service on day 1
Thermoelectric 8 out of service on day 1
Thermoelectric 9 out of service on day 1
Thermoelectric 10 out of service on day 1
Thermoelectric 11 out of service on day 1
Thermoelectric 12 out of service on day 2
Thermoelectric 13 out of service on day 1
Thermoelectric 14 out of service on day 5
Thermoelectric 15 out of service on day 2
Thermoelectric 16 out of service on day 1
Thermoelectric 17 out of service on day 1
Thermoelectric 18 out of service on day 2
Thermoelectric 19 out of service on day 3
Thermoelectric 0 re-enter service on day 11
Thermoelectric 1 re-enter service on day 2
Thermoelectric 2 re-enter service on day 3
Thermoelectric 4 re-enter service on day

IndexError: list index out of range